In [14]:
import os
import re
import string
import subprocess
import shutil

In [15]:
with open("txt/utf/g1/19.txt", "r", encoding="macRoman") as file:
    lines = file.readlines()  # List of lines
print(lines)


['Si Maya ay pangalawa sa tatlong magkakapatid. Nakatira sila sa abang tahanan ng Barangay San Isidro, lungsod ng Heneral Santos. Mapagmahal ang kanyang mga magulang na sina Mang Mando at Aling Memay. Mahilig maglaro si Maya sa kanyang manika, na bigay ng kanyang Ante Anna. Araw ng Sabado, naglalaro ang magkakapatid, ‚ÄúKuya Miguel, maglalaro tayo ng manika ko‚Äù, tawag ni Maya sa kanyang nakatatandang kapatid. Bog,Bog,Bog ‚ÄúSandali lamang naglalaro pa kami ni Marco ng basketball‚Äù, tugon naman ng kanyang Kuya Miguel. Si Marco ay ang kanilang nakababatang kapatid. Mag-isa na lamang naglaro si Maya sa kanyang manika na pinangalanan niyang Meya. ‚ÄúMmmmm‚Ä¶.mmmmm, tulog na!tulog na! Meya kong mahal‚Äù. Awit ni Maya habang inuugoy ang kanyang manika. Inilapag niya ito sa loob ng kahon bilang kama ng kanyang manika. Pumunta si Maya sa kusina upang uminom ng tubig. Nakita niya sa labas ang dalawa niyang kapatid na naglalaro ng bola. ‚ÄúKuya Miguel, ipasa mo rin sa akin ang bola‚Äù, nakang

## Normalize UTF

In [16]:
# Change grade level for each directory, use "gtest" for testing
input_folder = "txt/utf/gtest/" 
output_folder = "txt/normalized/gtest/"

# Define a function for text normalization
def preprocess_text(text):
    # WARNING: Removing uppercase would confuse tagger for proper nouns
    # text = text.lower()
    text = text.replace("“", "").replace("”", "").replace(",", "")
    text = text.replace("…",".")
    text = text.replace("..", ".")
    text = text.replace(".", "!").replace("?", "!").replace("!", "!")
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Loop through all files in the input directory
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):  # Process only text files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Read the original text file
        with open(input_path, "r", encoding="utf-8") as file:
            content = file.read()

        # Normalize the text
        cleaned_text = preprocess_text(content)

        # Save the normalized text to the new folder
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(cleaned_text)

        print(f"Processed: {input_path} -> {output_path}")

Processed: txt/utf/gtest/19.txt -> txt/normalized/gtest/19.txt


## Parts of Speech Tagger

In [ ]:
# Change grade level for each directory, use "gtest" for testing
input_folder = "txt/normalized/gtest"
output_folder = "txt/tagged/gtest"

# Directory containing the Stanford POS Tagger
stanford_dir = "stanford-postagger-full-2020-11-17/"
stanford_input = "hold-input.txt"

def copy_to_stanford(source_file):
    destination_file = "stanford-postagger-full-2020-11-17/hold-input.txt"

    shutil.copyfile(source_file, destination_file)

# source_file = "txt/normalized/g1/19.txt"
# input_filename = os.path.splitext(os.path.basename(source_file))[0]
# output_file_path = os.path.join(output_folder, f"{input_filename}-tagged.txt")

for source_file in os.listdir(input_folder):
    if source_file.endswith(".txt"):
        # Naming convention to sustain original indexing
        input_filename = os.path.splitext(os.path.basename(source_file))[0]
        # output_file_path = os.path.join(output_folder, f"{input_filename}-tagged.txt")
        output_file_path = os.path.join(output_folder, f"{input_filename}.txt")
        
        # Copy source file inside stanford directory
        copy_to_stanford(os.path.join(input_folder, source_file))

        # Java command to begin POS
        cmd = [
            "java", "-mx300m",
            "-classpath", "stanford-postagger.jar",
            "edu.stanford.nlp.tagger.maxent.MaxentTagger",
            "-model", "models/filipino-left5words-owlqn2-distsim-pref6-inf2.tagger",
            "-textFile", stanford_input
        ]

        with open(output_file_path, "w") as output_file:
            subprocess.run(cmd, stdout=output_file, stderr=subprocess.PIPE, text=True, cwd=stanford_dir)
        print(f"Tagged: {source_file} -> {output_file_path}")
